In [79]:
from ase.io import read,write
import numpy as np
atoms=read('gulp.xyz',format='xyz')
atoms.positions
atoms.set_cell([[21.889317,0,0], [0,21.889317,0],[0,0,21.889317]])

In [80]:
write('optimized.vasp',atoms,format='vasp',vasp5=True)

In [81]:
#check nearest distance
natom=len(atoms.positions)
dist=np.zeros((natom,natom,3))
from tqdm import tqdm
from nearest import find_nearest
for i in tqdm(range(natom)):  
    for j in range(i):
        xdc,ydc,zdc,rmin=find_nearest(atoms,i,j)
        dist[i,j]=np.array([xdc,ydc,zdc])

100%|██████████| 512/512 [01:08<00:00,  7.44it/s]


In [82]:
GULP_distx=np.loadtxt('distx.dat')[:,2]
GULP_disty=np.loadtxt('disty.dat')[:,2]
GULP_distz=np.loadtxt('distz.dat')[:,2]

In [83]:
GULP_distx

array([10.14206505,  7.88180726, -2.26025778, ..., -9.25122951,
       -7.09069481, -2.65821372])

In [84]:
from tqdm import tqdm
counter=0
diff=[]
for i in tqdm(range(natom)):  
    for j in range(i):
        diffx=np.abs(dist[i,j,0]-GULP_distx[counter])
        diffy=np.abs(dist[i,j,1]-GULP_disty[counter])
        diffz=np.abs(dist[i,j,1]-GULP_disty[counter])
        diff.append([diffx,diffy,diffz])
        #print(diffx,diffy,diffz)
        counter+=1

100%|██████████| 512/512 [00:00<00:00, 632.71it/s]


In [85]:
np.max(np.array(diff))

9.980016812960457e-10

距離の計算は間違ってないように見える

In [86]:
atoms.get_masses()

array([28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
      

In [87]:
import numpy as np
from ase.io import read

'''
evaluate velocity operator.
structure file: unitcell structure with vasp POSCAR format
FC_file: force constant file of phonopy format 
'''
def get_Vij(structure_file,FC_file):
    atoms=read(structure_file,format='vasp')
    masses=atoms.get_masses()
    natom=len(atoms.positions)

    dist=np.zeros((natom,natom,3))

    from nearest import find_nearest
    for i in range(natom):  
        for j in range(i):
            xdc,ydc,zdc,rmin=find_nearest(atoms,i,j)
            dist[i,j]=np.array([xdc,ydc,zdc])
    
    #reading phonopy format force constant
    with open(FC_file,'r') as fc:
        lines=fc.readlines()

        nlineblock=4
        fc_all=np.zeros((natom,natom,3,3))
        start=1
        for i in range(natom):
            for j in range(natom):
                fc_block=lines[start+1:start+nlineblock]
                fc=np.loadtxt(fc_block)
                fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
                #fc_all[i,j]=fc
                start=start+nlineblock
    
    Rx=dist[:,:,0]
    Ry=dist[:,:,1]
    Rz=dist[:,:,2]  

    Vx=np.zeros((natom,natom,3,3))
    Vy=np.zeros((natom,natom,3,3))
    Vz=np.zeros((natom,natom,3,3))

    #loop only for lower triangle
    #make sure that Vij=-Vji
    for i in range(natom):
        for j in range(i):
            Vijx=-Rx[i,j]*fc_all[i,j]
            Vx[i,j]=Vijx
            Vx[j,i]=-Vijx.T

            Vijy=-Ry[i,j]*fc_all[i,j]
            Vy[i,j]=Vijy
            Vy[j,i]=-Vijy.T

            Vijz=-Rz[i,j]*fc_all[i,j]
            Vz[i,j]=Vijz
            Vz[j,i]=-Vijz.T

    #reshape to (natom*3,natom*3) that matches with GULP format

    flatVx=np.reshape(Vx.transpose(0,2,1,3),(natom*3,natom*3))
    flatVy=np.reshape(Vy.transpose(0,2,1,3),(natom*3,natom*3))
    flatVz=np.reshape(Vz.transpose(0,2,1,3),(natom*3,natom*3))

    return flatVx, flatVy, flatVz

In [88]:
structure_file='optimized.vasp'
FC_file='FORCE_CONSTANTS_2ND'

Vx, Vy, Vz=get_Vij(structure_file,FC_file)

In [89]:
import numpy as np 
natom=len(atoms.positions)
eigenvector=np.loadtxt('eigr.dat').reshape((natom*3,natom*3))
omega=np.loadtxt('freq.dat')

In [90]:
'''
evaluate heat flux operator matrix element.
Vx, Vy, Vz is the return of get_Vij
omega--> phonon frequency
note that eigenvector is assumed to store in column order (same as the return of numpy.linalg.eig)
'''
def get_Sij(Vx,Vy,Vz, eigenvector, omega):

    nmodes=len(omega)

    #confirm matrix shape
    if(Vx.shape[0]!=Vx.shape[1] or Vx.shape[0]!=nmodes):
        assert "matrix shape Vx is strange"

    if(Vy.shape[0]!=Vy.shape[1] or Vy.shape[0]!=nmodes):
        assert "matrix shape Vy is strange"   

    if(Vz.shape[0]!=Vz.shape[1] or Vz.shape[0]!=nmodes):
        assert "matrix shape Vz is strange"  

    if(eigenvector.shape[0]!=eigenvector.shape[1] or eigenvector.shape[0]!=nmodes):
        assert "matrix shape eigenvector is strange" 

    #here the shape of eigenvector is assumed to that the typical return of numpy.linalg.eig
    #thus, each column is the eigenvector of each mode
    tmpx=np.dot(Vx,eigenvector)
    EVijx=np.dot(eigenvector.T,tmpx)

    tmpy=np.dot(Vy,eigenvector)
    EVijy=np.dot(eigenvector.T,tmpy)

    tmpz=np.dot(Vz,eigenvector)
    EVijz=np.dot(eigenvector.T,tmpz)


    Sijx=np.zeros((nmodes,nmodes))
    Sijy=np.zeros((nmodes,nmodes))
    Sijz=np.zeros((nmodes,nmodes))

    nfreqmin = 0
    inv_omega=np.zeros(nmodes)
    for i in range(nmodes):
        if (nfreqmin==0 and omega[i]> 0.01):
            nfreqmin = i
        if omega[i] >0.0:
            inv_omega[i]=1.0/np.sqrt(omega[i])
        else:
            inv_omega[i]=0.0


    for i in range(nmodes):
        for j in range(nmodes):
            Sijx[i,j]=EVijx[i,j]*(omega[i]+omega[j])*inv_omega[i]*inv_omega[j]
            Sijy[i,j]=EVijy[i,j]*(omega[i]+omega[j])*inv_omega[i]*inv_omega[j]
            Sijz[i,j]=EVijz[i,j]*(omega[i]+omega[j])*inv_omega[i]*inv_omega[j]
    
    return Sijx, Sijy, Sijz

In [91]:
Sx,Sy,Sz=get_Sij(Vx,Vy,Vz,eigenvector,omega)

In [92]:
#check
GULP_Vijx=np.loadtxt('Vijx.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vx-GULP_Vijx) > 0.1)

(array([], dtype=int64), array([], dtype=int64))

In [93]:
Vx[0,243:260]

array([-0.11037678,  0.0387957 ,  0.13679122,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
       -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

In [94]:
GULP_Vijx[0,243:260]

array([-0.11035713,  0.0387888 ,  0.13676687,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

0,82の要素を詳しくみてみる

In [95]:
masses=atoms.get_masses()

with open(FC_file,'r') as fc:
    lines=fc.readlines()
    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
            #fc_all[i,j]=fc
            start=start+nlineblock

In [96]:
fc_all[0,81]

array([[-0.10056151,  0.03534579,  0.12462704],
       [ 0.05026058, -0.08582116, -0.14422731],
       [ 0.10238973, -0.09820934, -0.25153559]])

In [97]:
fc_all[81,0]

array([[-0.10056151,  0.05026058,  0.10238973],
       [ 0.03534579, -0.08582116, -0.09820934],
       [ 0.12462704, -0.14422731, -0.25153559]])

アモルファスの場合は非対角項がある。i,jとj,iは転置の関係にある。
これが上手く反映されていないように見える

In [98]:
dist[81,0,0]

1.0976046420000003

In [99]:
fc_all[0,81]*dist[81,0,0]

array([[-0.11037678,  0.0387957 ,  0.13679122],
       [ 0.05516625, -0.0941977 , -0.15830457],
       [ 0.11238344, -0.10779503, -0.27608664]])

```
-0.11035713,  0.0387888 ,  0.13676687
```
force constant と距離の積から求めると、GULPの計算とあっている。
tensorのreshapeの挙動がおかしいのかも
--> 転置をきちんと入れる処理で解決

In [101]:
#check
GULP_Vijy=np.loadtxt('Vijy.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vy-GULP_Vijy) > 0.01)

(array([], dtype=int64), array([], dtype=int64))

In [102]:
GULP_Vijz=np.loadtxt('Vijz.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vz-GULP_Vijz) > 0.01)

(array([], dtype=int64), array([], dtype=int64))

In [103]:
GULP_Sijx=np.loadtxt('Sijx.dat').reshape((natom*3,natom*3))
np.where(np.abs(Sx-GULP_Sijx) > 0.01)

(array([], dtype=int64), array([], dtype=int64))

In [ ]:
GULP_Sijy=np.loadtxt('Sijy.dat').reshape((natom*3,natom*3))
np.where(np.abs(Sy-GULP_Sijy) > 0.01)